In [1]:
import numpy as np
import pandas as pd
%config IPCompleter.greedy=True
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

## Создание csv с данными о типах блюд в чеках по дням

#### Загрузим тренировочные данные

In [2]:
train = pd.read_csv('data/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919985 entries, 0 to 919984
Data columns (total 6 columns):
chknum       919985 non-null object
person_id    919985 non-null int64
month        919985 non-null int64
day          919985 non-null int64
good         919985 non-null object
good_id      919985 non-null int64
dtypes: int64(4), object(2)
memory usage: 42.1+ MB


In [3]:
train.head()

,chknum,person_id,month,day,good,good_id
0,id0,1,1,17,Грибовница со взбитыми яйцами 250гр,12
1,id0,1,1,17,Булочка с сыром 60гр,13
2,id0,1,1,17,Салат бар 1кг,8
3,id0,1,1,17,С-т из огур/помидор/перец/масло 120гр,124
4,id1,626,1,17,Индейка/грибы/сыр 80/20гр,181


#### Меню

Загрузим наше меню

In [4]:
menu_file = pd.read_csv('data/menu_tagged.csv')

In [5]:
menu_file.columns

Index(['good_id', 'good', 'first_course', 'main_course', 'drink', 'bakery',
       'dessert', 'snacks', 'salad', 'meat&fish', 'fried', 'sweet', 'salty',
       'spicy', 'sour', 'lenten', 'meat', 'seafoods', 'pepper', 'mushrooms',
       'vegetables', 'fruits', 'milk', 'croups', 'eggs', 'pasta', 'flour',
       'onion_or_garlic', 'other'],
      dtype='object')

Сделаем доступным обращение к данным о типах блюда по его id

In [6]:
menu = {}
for row in menu_file.values:
    menu[int(row[0])] = list(map(int, row[2:10]))
print(menu[1])

[0, 1, 0, 0, 0, 0, 0, 0]


Можно представить наши данные о людях в виде такой вложенной структуры (можно ввести классы Date, Person, Check)

In [7]:
# chknum person_id month day good good_id
# dates: dict of dicts of checks and persons with lists of goods
dates = {}
for row in train.values:
    chknum, person_id, month, day, good, good_id = row
    current_date = (month, day)
    dates.setdefault(current_date, dict()).setdefault(chknum, (person_id, list()))[1].append((good, good_id))
# dates[(month, day)]['chk_id'][1] - list of goods
# dates[(month, day)]['chk_id'][0] - person_id
print(dates[(1, 17)]['id0'][1])

[('Грибовница со взбитыми яйцами 250гр', 12), ('Булочка с сыром 60гр', 13), ('Салат бар 1кг', 8), ('С-т из огур/помидор/перец/масло 120гр', 124)]


Далее можно собрать статистику во вложенную структуру

In [8]:
# stats: dict of lists  representing (person, types_counts_list) by date
stats = {}
for date, date_val in dates.items():
    onday = []
    for chk in date_val.values():
        buy = (chk[0], [0] * 8)
        for good in chk[1]:
            for i in range(8):
                buy[1][i] += menu[good[1]][i]
        onday.append(buy)
    stats[date] = onday

# print(stats[(1, 17)])

Или же в виде таблицы

In [9]:
columns = ['month', 'day', 'chknum', 'person_id', 'first_course', 'main_course', 'drink', 'bakery', 'dessert', 'snacks', 'salad', 'meat&fish']

data_for_pd = []
for date, date_val in dates.items():
    for chknum, chk in date_val.items():
        buy = [0] * 8
        for good in chk[1]:
            for i in range(8):
                buy[i] += menu[good[1]][i]
        data_for_pd.append([date[0], date[1], chknum, chk[0], *buy])

df = pd.DataFrame(data_for_pd, columns=columns)
df.head()

,month,day,chknum,person_id,first_course,main_course,drink,bakery,dessert,snacks,salad,meat&fish
0,1,17,id0,1,1,0,0,1,0,0,2,0
1,1,17,id1,626,0,1,1,0,0,0,2,1
2,1,17,id2,1438,0,1,1,0,0,0,2,1
3,1,17,id3,190,1,1,1,0,0,3,1,1
4,1,17,id4,139,0,1,1,0,0,0,1,1


In [10]:
df.to_csv('stats/goods_types_by_day_and_chk.csv', index=False)

## Анализ